In [2]:
!pip install pinecone-text pinecone-client pinecone-notebooks

  Using cached wget-3.2-py3-none-any.whl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 1.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 506.4 kB/s eta 0:00:00 0:00:01

[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [4]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [5]:
from pinecone import Pinecone, ServerlessSpec
index_name = "hybrid-search-pinecone-langchain"

pc = Pinecone(api_key=api_key)

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name = index_name,
        dimension = 384,
        metric = "dotproduct",
        spec = ServerlessSpec(cloud = "aws", region = "us-east-1")
    )

In [6]:
index = pc.Index(index_name)
index

In [7]:
import os
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")
embeddings

/home/ashwinth396/ML projects/langchain learn/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [8]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder().default()
bm25_encoder

[nltk_data] Downloading package punkt to
[nltk_data]     /home/ashwinth396/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ashwinth396/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
sentences = ["In 2023, I visited Paris", "In 2022, I visited New York", "In 2021, I visited New Orleans"]

bm25_encoder.fit(sentences)
bm25_encoder.dump("bm25_values.json")

100%|██████████| 3/3 [00:00<00:00, 559.27it/s]


In [15]:
bm25_encoder = BM25Encoder.load(self= "bm25_values.json")

TypeError: BM25Encoder.load() missing 1 required positional argument: 'path'

In [16]:
retriever = PineconeHybridSearchRetriever(embeddings=embeddings, sparse_encoder= bm25_encoder, index = index)

In [17]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x7d12dd93d510>, index=<pinecone.data.index.Index object at 0x7d13d6831810>)

In [19]:
retriever.add_texts(sentences)

100%|██████████| 1/1 [00:13<00:00, 13.86s/it]


In [20]:
retriever.invoke("What city did I visit recently?")

[Document(page_content='In 2021, I visited New Orleans'),
 Document(page_content='In 2022, I visited New York'),
 Document(page_content='In 2023, I visited Paris')]